#### 1. Import Dependencies

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import os
import ast
import random
import shutil

from tqdm import tqdm_notebook

import xml.etree.ElementTree as xml

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO

#### 2. Define Paths

In [ ]:
img_train = os.path.join("Yolo-Data/train/images/")
lbl_train = os.path.join("Yolo-Data/train/labels/")
img_test = os.path.join("Yolo-Data/val/images/")
lbl_test = os.path.join("Yolo-Data/val/labels/")
xml_path = os.path.join("Yolo-Data/xml/")

#### 3. Use the class below to load data

In [ ]:
class DataLoader:

    def load_image(path):
        images = os.listdir(path)
        images = sorted(images)

        shapes = []
        for file in images:
            full_path = path+file
            img = cv.imread(full_path)
            shapes.append(img.shape)
            img = cv.resize(img, (640,640))
            cv.imwrite(full_path, img)

        return(shapes)
    
    def load_xml(path, save_path, shapes):
        
        xmls = os.listdir(path)
        xmls = sorted(xmls)
        i = 0
        for file in xmls:
            try:
                xml_file = xml.parse(path+file)
                root = xml_file.getroot()
            
                xmin = int(float(root.find(".//xmin").text))
                xmax = int(float(root.find(".//xmax").text))
                ymin = int(float(root.find(".//ymin").text))
                ymax = int(float(root.find(".//ymax").text))
                
                label_id = 0
                x = ((xmin+xmax)/2)/shapes[i][1]
                y = ((ymin+ymax)/2)/shapes[i][0]
                width = (abs(xmin-xmax))/shapes[i][1]
                height = (abs(ymin-ymax))/shapes[i][0]
                
                yolo_label = [0, x, y, width, height]
                with open(f"{save_path+(file.split('.')[0])}.txt", "w") as f:
                    f.write(f"{label_id} {x} {y} {width} {height}")
    
                i += 1

            except xml.ParseError as e:
                print(f"XML ParseError in {path, file}: {e}")
            except FileNotFoundError:
                print(f"File not found: {path}")
            except Exception as e:
                print(f"Unexpected error: {e}")

#### 4. Set the config of 'Yolo Model' based on what you need

In [ ]:
# Configuring .yaml file for the pathes of each class for training the YOLO model
with open("setting.yaml", 'w') as f:
    f.write("""
train: Yolo-Data/train/images
val: Yolo-Data/val/images
nc: 1
names: ['Plate']
""")

# It's possible to use torch to load YOLO model but i recommend you to use it the same way i did
model = YOLO("yolov5s.pt")
results = model.train(data="setting.yaml", epochs=3, imgsz=640)